In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
def standard_units(x):
    """Convert array x to standard units."""
    mean = np.mean(x)
    std = np.std(x)
    return (x - mean) / std

(back to slides)
# Linear Regression

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in this class


def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_table(r):
    """
    Generate a table of 1000 x,y data points in standard units
    whose correlation is approximately equal to r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

Let's look at an example where our data is linearly correlated

In [ ]:
example = r_table(0.99)
example.show(3)

`example` is a table of 1k examples where `x` and `y` have a correlation coefficient of 0.99



**Question:** How could we visualize the correlation?

<details>
<summary>Solution</summary>
  example.scatter('x', 'y')
</details>

In [ ]:
example.scatter('x', 'y')
resize_window()

Let's now make our predictions

In [ ]:
def nn_prediction_example(x_val):
    """ Predicts y-value for x based on the example table """
    neighbors = example.where('x', are.between(x_val - .25, x_val + .25))
    return np.mean(neighbors.column('y'))

**Question:** What should our `y` value be when `x` is 0?

In [ ]:
nn_prediction_example(0)

**Question** What about when `x` is 2 or -2?

In [ ]:
nn_prediction_example(2), nn_prediction_example(-2)

Now let's apply the prediction function to our table

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example

Let's plot the predictions (in yellow)

In [ ]:
example.scatter('x')
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, color='dodgerblue')
resize_window()

Now let's make a new table where the data is not linearly correlated

In [ ]:
example = r_table(0)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=0, color='dodgerblue')
resize_window()

Now let's look at an example where the scatter plot is oval shaped

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

(back to slides)

# Linear regression: defining the line


In [ ]:
# Copy-pasted from above
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

**Question:** let's compute the slope and intercept

<details>
<summary>Equation</summary>
  ![image.png](slope_intercept_eq.png)
</details>


<!-- copy the equation markdown to this cell and change the cell to markdown -->
(put equation here)

In [ ]:
def slope(t, x, y):
    # r * standard_dev(y) / standard_dev(x)
    x_array = t.column(x)
    y_array = t.column(y)
    r = correlation(t, x, y)
    
    return r * np.std(y_array) / np.std(x_array)
    

def intercept(t, x, y):
    # mean(y) - slope * mean(x)
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y')

In [ ]:
intercept(example, 'x', 'y')

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=0.5, color='dodgerblue')
resize_window()

(back to slides)

# Root Mean Squared Error

In [ ]:
little_women = Table().read_table('https://www.inferentialthinking.com/data/little_women.csv')
little_women

In [ ]:
little_women.scatter(1,0)

**Question:** Do we think there is a linear association here?

In [ ]:
correlation(little_women, 'Periods', 'Characters')

**Question:** Let's compute the correlation

In [ ]:
correlation(little_women, 1, 0)

Now lets predict the number of characters based on the number of periods in a chapter

In [ ]:
pred_characters = fitted_values(little_women, 'Periods', 'Characters') #
pred_characters 

In [ ]:
little_women_fitted = little_women.with_columns("fitted", pred_characters)
#little_women_fitted.hist() # What does this histogram mean? 
# Then lets uncomment the next line to look at the scatter of the predictions
little_women_fitted.scatter(1)

## Squared Error

This function will draw a line with a specified slope and intercept and will draw red lines showing the errors

In [ ]:
sample = [[131, 14431], [231, 20558], [392, 40935], [157, 23524]]
def lw_errors(slope, intercept):
    print("Slope:      ", np.round(slope), 'characters per period')
    print("Intercept:  ", np.round(intercept), 'characters')
    little_women.scatter('Periods', 'Characters')
    xlims = np.array([50, 450])
    plots.plot(xlims, slope * xlims + intercept, lw=2)
    for x,y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=2)

In [ ]:
lw_errors(50, 1000)

In [ ]:
lw_errors(100, 1000)

Let's try it with more slopes and intercepts

In [ ]:
lw_errors(70, 1000)

Lets make a line that goes through part of the scatter diagram

Now let's compute the root mean square error for the differnt lines we just considered


<details>
<summary>Solution Prediction</summary>
  slope * x + intercept
</details>


<details>
<summary>Solution MSE</summary>
  np.mean((y - prediction) ** 2)
</details>


In [ ]:
def lw_rmse(slope, intercept):
    lw_errors(slope, intercept)
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    prediction = ...
    mse = ...
    print("Root mean squared error:", round(mse ** 0.5, 2))

In [ ]:
lw_rmse(50,1000)

In [ ]:
lw_rmse(70,1000)

In [ ]:
lw_rmse(100,1000)

In [ ]:
lw_rmse(102,1000)

In [ ]:
lw_rmse(110,1000)

In [ ]:
lw_rmse(105,1000)

Let's comput the rmse for the regression line 

<details>
<summary>Solution</summary>
  lw_rmse(slope(little_women, 1, 0), intercept(little_women, 1, 0))
</details>

The regression line is the line that minimzies the root mean squared error.

(back to slides)
# Least Squares

In [ ]:
x = np.arange(1, 3, 0.1)
y = (x-2)**2 + 3
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
def f(x):
    return ((x-2)**2) + 3

What x-value gives us the smalles y-value?

In [ ]:
minimize(f)

In [ ]:
x = np.arange(-1.5, 1.5, 0.05)
y2 = 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 
Table().with_columns('x', x, 'y', y2).plot('x')

In [ ]:
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4

What x-value gives us the smalles y-value?

In [ ]:
minimize(complicated_function)

(back to slides)
# Minimizing MSE


In [ ]:
def lw_mse(any_slope, any_intercept):
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    estimate = any_slope*x + any_intercept
    return (np.mean((y - estimate) ** 2)) 

In [ ]:
lw_mse(1500, 20000), lw_mse(1500, 20000)

In [ ]:
minimize(lw_mse)

In [ ]:
slope(little_women, "Periods", "Characters"), intercept(little_women, "Periods", "Characters")

(back to slides)
# Residuals 

Let's make a new function called residuals

<details>
<summary>Solution</summary>
  t.column(y) - predictions
</details>


In [ ]:
def residuals(t, x, y):
    """ Returns residual for each prediction, 
        i.e. the difference between the true y and predicted y"""
    predictions = fitted_values(t, x, y)
    return ...

In [ ]:
residuals(little_women, "Periods", "Characters")

Let's add residuals to a table and plot the table
    <details>
<summary>Solution</summary>
  little_women_fitted.with_columns('residuals', residuals(little_women, "Periods", "Characters")).scatter('Periods')
</details>

In [ ]:
  little_women_fitted.with_columns('residuals', residuals(little_women, "Periods", "Characters")).scatter('Periods')

**Question:** Why are the residuals on the bottom?
        <details>
<summary>Solution</summary>
  Becuase residuals show the difference between the prediction and the true value
</details>

Let's plot the residuals and the predictions seperately

In [ ]:
def plot_residuals(t, x, y):
    with_residuals = t.with_columns(
        "Fitted", fitted_values(t, x, y),
        "Residual", residuals(t, x, y)/ 1000 # I did this division just for this example
    )
    with_residuals.select(x, y, 'Fitted').scatter(0)
    with_residuals.scatter(x, 'Residual')

In [ ]:
plot_residuals(little_women, "Periods", "Characters")

We see the residuals clustered around 0. This makes sense because the data is correlated

(back to demo)
# Nonlinear Regression

We will look at two examples


In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
correlation(shotput, "Weight Lifted", "Shot Put Distance")

In [ ]:
shotput.scatter('Weight Lifted')

**Question**: Is the association linear?

In [ ]:
plot_residuals(shotput, "Weight Lifted", "Shot Put Distance")

Let's look at another example

In [ ]:
height_weight = Table.read_table('us_women.csv')
height_weight

**Question**: Is the association linear?

In [ ]:
correlation(height_weight, 'height', 'ave weight')

In [ ]:
plot_residuals(height_weight, 'height', "ave weight")

(back to slides) 
# A Measure of cluster

In [ ]:
def plot_fitted(t, x, y):
    tbl = t.select(x, y)
    tbl.with_columns('Fitted Value', fitted_values(t, x, y)).scatter(0)

In [ ]:
plot_fitted(height_weight, 'height', "ave weight")

In [ ]:
weight_pred_sd = np.std(fitted_values(height_weight, 'height', 'ave weight'))
weight_observed_sd = np.std(height_weight.column('ave weight'))
print(weight_pred_sd)
print(weight_observed_sd)

In [ ]:
weight_pred_sd / weight_observed_sd

In [ ]:
correlation(height_weight, 'height', 'ave weight')